***GENERATED CODE FOR visaclass PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.ml.feature import Binarizer
from pyspark.sql.functions import round
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


def BinarizerTransform(df, params, transformationData={}):
    dfReturn = df
    transform_params = params
    feature = transform_params['feature']
    outcol = feature + "_binarizer"
    dfReturn = dfReturn.withColumn("feature_cast", dfReturn[feature].cast("double")).drop(feature)\
        .withColumnRenamed("feature_cast", feature)

    dfReturn = dfReturn.fillna({feature: 0.0})
    binarizer = Binarizer(threshold=float(
        transformationData['threshold']), inputCol=feature, outputCol=outcol)
    binarizedDataFrame = binarizer.transform(dfReturn)

    # binarizedDataFrame=binarizedDataFrame.drop(feature).withColumnRenamed(outcol,feature)

    dfReturn = binarizedDataFrame
    dfReturn = dfReturn.withColumn(feature, round(dfReturn[feature], 2))

    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'gender', 'transformation_label': 'String Indexer'}], 'feature': 'gender', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '500', 'mean': '', 'stddev': '', 'min': 'female', 'max': 'male', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'gender'}, {'feature_label': 'gender', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('gender')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'employer_name', 'transformation_label': 'String Indexer'}], 'feature': 'employer_name', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '4C Air, Inc.', 'max': 'iSpace, Inc.', 'missing': '0', 'distinct': '311'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'employer_name'}, {'feature_label': 'employer_name', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('employer_name')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'mail_addr', 'transformation_label': 'String Indexer'}], 'feature': 'mail_addr', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '1 Amgen Center Dr', 'max': 'One Penn Plaza', 'missing': '0', 'distinct': '304'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'mail_addr'}, {'feature_label': 'mail_addr', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('mail_addr')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'city', 'transformation_label': 'String Indexer'}], 'feature': 'city', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Albuquerque', 'max': 'Woodland Hls', 'missing': '0', 'distinct': '183'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'city'}, {'feature_label': 'city', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('city')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'state', 'transformation_label': 'String Indexer'}], 'feature': 'state', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'AL', 'max': 'WI', 'missing': '0', 'distinct': '38'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'state'}, {'feature_label': 'state', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('state')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'zip', 'transformation_label': 'String Indexer'}], 'feature': 'zip', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '52250.97', 'stddev': '36131.99', 'min': '01545-3415', 'max': '98121-2770', 'missing': '0', 'distinct': '293'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'zip'}, {'feature_label': 'zip', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('zip')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'status_type', 'transformation_label': 'String Indexer'}], 'feature': 'status_type', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'SELECTED', 'max': 'SELECTED', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'status_type'}, {'feature_label': 'status_type', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('status_type')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'rec_date', 'transformation_label': 'String Indexer'}], 'feature': 'rec_date', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '10/1/2020', 'max': '9/30/2020', 'missing': '0', 'distinct': '91'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'rec_date'}, {'feature_label': 'rec_date', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('rec_date')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'FIRST_DECISION', 'transformation_label': 'String Indexer'}], 'feature': 'FIRST_DECISION', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '499', 'mean': '', 'stddev': '', 'min': 'Approved', 'max': 'Denied', 'missing': '1', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'FIRST_DECISION'}, {'feature_label': 'FIRST_DECISION', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('FIRST_DECISION')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'first_decision_date', 'transformation_label': 'String Indexer'}], 'feature': 'first_decision_date', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '499', 'mean': '', 'stddev': '', 'min': '1/13/2021', 'max': '9/9/2020', 'missing': '1', 'distinct': '173'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'first_decision_date'}, {'feature_label': 'first_decision_date', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('first_decision_date')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'REQUESTED_CLASS', 'transformation_label': 'String Indexer'}], 'feature': 'REQUESTED_CLASS', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '1B1', 'max': '1B1', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'REQUESTED_CLASS'}, {'feature_label': 'REQUESTED_CLASS', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('REQUESTED_CLASS')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'BASIS_FOR_CLASSIFICATION', 'transformation_label': 'String Indexer'}], 'feature': 'BASIS_FOR_CLASSIFICATION', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'A', 'max': 'B', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'BASIS_FOR_CLASSIFICATION'}, {'feature_label': 'BASIS_FOR_CLASSIFICATION', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(
            'BASIS_FOR_CLASSIFICATION')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'REQUESTED_ACTION', 'transformation_label': 'String Indexer'}], 'feature': 'REQUESTED_ACTION', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'A', 'max': 'C', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'REQUESTED_ACTION'}, {'feature_label': 'REQUESTED_ACTION', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('REQUESTED_ACTION')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'BEN_COUNTRY_OF_BIRTH', 'transformation_label': 'String Indexer'}], 'feature': 'BEN_COUNTRY_OF_BIRTH', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'ARGEN', 'max': 'VENEZ', 'missing': '0', 'distinct': '40'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'BEN_COUNTRY_OF_BIRTH'}, {'feature_label': 'BEN_COUNTRY_OF_BIRTH', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('BEN_COUNTRY_OF_BIRTH')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'BEN_CURRENT_CLASS', 'transformation_label': 'String Indexer'}], 'feature': 'BEN_CURRENT_CLASS', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'B1', 'max': 'UU', 'missing': '0', 'distinct': '16'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'BEN_CURRENT_CLASS'}, {'feature_label': 'BEN_CURRENT_CLASS', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('BEN_CURRENT_CLASS')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'JOB_TITLE', 'transformation_label': 'String Indexer'}], 'feature': 'JOB_TITLE', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '2D ARTIST', 'max': 'VP ENTERPRISE IMPROVEMENT', 'missing': '0', 'distinct': '188'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'JOB_TITLE'}, {'feature_label': 'JOB_TITLE', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('JOB_TITLE')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'WORKSITE_ZIP', 'transformation_label': 'String Indexer'}], 'feature': 'WORKSITE_ZIP', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '9156623.82', 'stddev': '85021794.1', 'min': '10001', 'max': '98121', 'missing': '0', 'distinct': '305'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'WORKSITE_ZIP'}, {'feature_label': 'WORKSITE_ZIP', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('WORKSITE_ZIP')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'valid_from', 'transformation_label': 'String Indexer'}], 'feature': 'valid_from', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '484', 'mean': '', 'stddev': '', 'min': '1/11/2021', 'max': '6/24/2021', 'missing': '16', 'distinct': '73'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'valid_from'}, {'feature_label': 'valid_from', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('valid_from')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'valid_to', 'transformation_label': 'String Indexer'}], 'feature': 'valid_to', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '475', 'mean': '', 'stddev': '', 'min': '1/28/2023', 'max': '9/9/2023', 'missing': '16', 'distinct': '81'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'valid_to'}, {'feature_label': 'valid_to', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('valid_to')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'BEN_EDUCATION_CODE', 'transformation_label': 'String Indexer'}], 'feature': 'BEN_EDUCATION_CODE', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'F', 'max': 'I', 'missing': '0', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'BEN_EDUCATION_CODE'}, {'feature_label': 'BEN_EDUCATION_CODE', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('BEN_EDUCATION_CODE')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ED_LEVEL_DEFINITION', 'transformation_label': 'String Indexer'}], 'feature': 'ED_LEVEL_DEFINITION', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': "BACHELOR'S DEGREE", 'max': 'PROFESSIONAL DEGREE', 'missing': '0', 'distinct': '6'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ED_LEVEL_DEFINITION'}, {'feature_label': 'ED_LEVEL_DEFINITION', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ED_LEVEL_DEFINITION')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'BEN_PFIELD_OF_STUDY', 'transformation_label': 'String Indexer'}], 'feature': 'BEN_PFIELD_OF_STUDY', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'ACCOUNTANCY', 'max': 'WEB DESIGN & NEW MEDIA', 'missing': '0', 'distinct': '214'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'BEN_PFIELD_OF_STUDY'}, {'feature_label': 'BEN_PFIELD_OF_STUDY', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('BEN_PFIELD_OF_STUDY')
        transformationDF = BinarizerTransform(transformationDF, {'transformationsData': [{'feature_label': 'BEN_COMP_PAID', 'threshold': 95916.516, 'transformation_label': 'Binarizer'}], 'feature': 'BEN_COMP_PAID', 'type': 'real', 'selected': 'True', 'replaceby': 'mean', 'stats': {
            'count': '500', 'mean': '96571.4', 'stddev': '35522.18', 'min': '26.91', 'max': '220000.0', 'missing': '0'}, 'transformation': [{'transformation': 'Binarizer', 'selectedAsDefault': 1}], 'updatedLabel': 'BEN_COMP_PAID'}, {'feature_label': 'BEN_COMP_PAID', 'threshold': 95916.516, 'transformation_label': 'Binarizer'})
        transformationDF = transformationDF.drop('BEN_COMP_PAID')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'CountryName', 'transformation_label': 'String Indexer'}], 'feature': 'CountryName', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'AUSTRALIA', 'max': 'ZIMBABWE', 'missing': '0', 'distinct': '40'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CountryName'}, {'feature_label': 'CountryName', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('CountryName')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Flag', 'transformation_label': 'String Indexer'}], 'feature': 'Flag', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '428', 'mean': '', 'stddev': '', 'min': 'https://flagcdn.com/w320/au.png', 'max': 'https://flagcdn.com/w320/zw.png', 'missing': '88', 'distinct': '38'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Flag'}, {'feature_label': 'Flag', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Flag')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=5,
                               population_size=15, use_dask=True)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run visaclassHooks.ipynb
try:
	#sourcePreExecutionHook()

	hb = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/H1B_2.csv', 'filename': 'H1B_2.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'dbfs_token': '', 'dbfs_domain': '', 'is_header': 'Use Header Line', 'server_url': '/nexusMax/NexusMaxPlatform/uploads/platform/', 'results_url': 'http://ml.colaberry.com:44040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run visaclassHooks.ipynb
try:
	#transformationPreExecutionHook()

	visaclassautofe = TransformationMain.run(hb,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "ben_year_of_birth", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1989.89", "stddev": "5.31", "min": "1962", "max": "1998", "missing": "0"}, "updatedLabel": "ben_year_of_birth"}, {"transformationsData": [{"feature_label": "gender", "transformation_label": "String Indexer"}], "feature": "gender", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "female", "max": "male", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "gender"}, {"transformationsData": [{"feature_label": "employer_name", "transformation_label": "String Indexer"}], "feature": "employer_name", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "4C Air, Inc.", "max": "iSpace, Inc.", "missing": "0", "distinct": "311"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "employer_name"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "FEIN", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "529524433.12", "stddev": "289979401.85", "min": "10526993", "max": "990080213", "missing": "0"}, "updatedLabel": "FEIN"}, {"transformationsData": [{"feature_label": "mail_addr", "transformation_label": "String Indexer"}], "feature": "mail_addr", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "1 Amgen Center Dr", "max": "One Penn Plaza", "missing": "0", "distinct": "304"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "mail_addr"}, {"transformationsData": [{"feature_label": "city", "transformation_label": "String Indexer"}], "feature": "city", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Albuquerque", "max": "Woodland Hls", "missing": "0", "distinct": "183"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "city"}, {"transformationsData": [{"feature_label": "state", "transformation_label": "String Indexer"}], "feature": "state", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "AL", "max": "WI", "missing": "0", "distinct": "38"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "state"}, {"transformationsData": [{"feature_label": "zip", "transformation_label": "String Indexer"}], "feature": "zip", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "52250.97", "stddev": "36131.99", "min": "01545-3415", "max": "98121-2770", "missing": "0", "distinct": "293"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "zip"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "lottery_year", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "2021.0", "stddev": "0.0", "min": "2021", "max": "2021", "missing": "0"}, "updatedLabel": "lottery_year"}, {"transformationsData": [{"feature_label": "status_type", "transformation_label": "String Indexer"}], "feature": "status_type", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "SELECTED", "max": "SELECTED", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "status_type"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "ben_multi_reg_ind", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.09", "stddev": "0.29", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "ben_multi_reg_ind"}, {"transformationsData": [{"feature_label": "rec_date", "transformation_label": "String Indexer"}], "feature": "rec_date", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "10/1/2020", "max": "9/30/2020", "missing": "0", "distinct": "91"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "rec_date"}, {"transformationsData": [{"feature_label": "FIRST_DECISION", "transformation_label": "String Indexer"}], "feature": "FIRST_DECISION", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "499", "mean": "", "stddev": "", "min": "Approved", "max": "Denied", "missing": "1", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "FIRST_DECISION"}, {"transformationsData": [{"feature_label": "first_decision_date", "transformation_label": "String Indexer"}], "feature": "first_decision_date", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "499", "mean": "", "stddev": "", "min": "1/13/2021", "max": "9/9/2020", "missing": "1", "distinct": "173"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "first_decision_date"}, {"transformationsData": [{"feature_label": "REQUESTED_CLASS", "transformation_label": "String Indexer"}], "feature": "REQUESTED_CLASS", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "1B1", "max": "1B1", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "REQUESTED_CLASS"}, {"transformationsData": [{"feature_label": "BASIS_FOR_CLASSIFICATION", "transformation_label": "String Indexer"}], "feature": "BASIS_FOR_CLASSIFICATION", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "A", "max": "B", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "BASIS_FOR_CLASSIFICATION"}, {"transformationsData": [{"feature_label": "REQUESTED_ACTION", "transformation_label": "String Indexer"}], "feature": "REQUESTED_ACTION", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "A", "max": "C", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "REQUESTED_ACTION"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "NUMBER_OF_BENEFICIARIES", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1.0", "stddev": "0.0", "min": "1", "max": "1", "missing": "0"}, "updatedLabel": "NUMBER_OF_BENEFICIARIES"}, {"transformationsData": [{"feature_label": "BEN_COUNTRY_OF_BIRTH", "transformation_label": "String Indexer"}], "feature": "BEN_COUNTRY_OF_BIRTH", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "ARGEN", "max": "VENEZ", "missing": "0", "distinct": "40"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "BEN_COUNTRY_OF_BIRTH"}, {"transformationsData": [{"feature_label": "BEN_CURRENT_CLASS", "transformation_label": "String Indexer"}], "feature": "BEN_CURRENT_CLASS", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "B1", "max": "UU", "missing": "0", "distinct": "16"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "BEN_CURRENT_CLASS"}, {"transformationsData": [{"feature_label": "JOB_TITLE", "transformation_label": "String Indexer"}], "feature": "JOB_TITLE", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "2D ARTIST", "max": "VP ENTERPRISE IMPROVEMENT", "missing": "0", "distinct": "188"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "JOB_TITLE"}, {"transformationsData": [{"feature_label": "WORKSITE_ZIP", "transformation_label": "String Indexer"}], "feature": "WORKSITE_ZIP", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "9156623.82", "stddev": "85021794.1", "min": "10001", "max": "98121", "missing": "0", "distinct": "305"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "WORKSITE_ZIP"}, {"transformationsData": [{"feature_label": "valid_from", "transformation_label": "String Indexer"}], "feature": "valid_from", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "484", "mean": "", "stddev": "", "min": "1/11/2021", "max": "6/24/2021", "missing": "16", "distinct": "73"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "valid_from"}, {"transformationsData": [{"feature_label": "valid_to", "transformation_label": "String Indexer"}], "feature": "valid_to", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "475", "mean": "", "stddev": "", "min": "1/28/2023", "max": "9/9/2023", "missing": "16", "distinct": "81"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "valid_to"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "NUM_OF_EMP_IN_US", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "28926.2", "stddev": "153461.89", "min": "0", "max": "2300000", "missing": "0"}, "updatedLabel": "NUM_OF_EMP_IN_US"}, {"transformationsData": [{"feature_label": "BEN_EDUCATION_CODE", "transformation_label": "String Indexer"}], "feature": "BEN_EDUCATION_CODE", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "F", "max": "I", "missing": "0", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "BEN_EDUCATION_CODE"}, {"transformationsData": [{"feature_label": "ED_LEVEL_DEFINITION", "transformation_label": "String Indexer"}], "feature": "ED_LEVEL_DEFINITION", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "BACHELOR'S DEGREE", "max": "PROFESSIONAL DEGREE", "missing": "0", "distinct": "6"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ED_LEVEL_DEFINITION"}, {"transformationsData": [{"feature_label": "BEN_PFIELD_OF_STUDY", "transformation_label": "String Indexer"}], "feature": "BEN_PFIELD_OF_STUDY", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "ACCOUNTANCY", "max": "WEB DESIGN & NEW MEDIA", "missing": "0", "distinct": "214"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "BEN_PFIELD_OF_STUDY"}, {"transformationsData": [{"feature_label": "BEN_COMP_PAID", "threshold": 95916.516, "transformation_label": "Binarizer"}], "feature": "BEN_COMP_PAID", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "96571.4", "stddev": "35522.18", "min": "26.91", "max": "220000.0", "missing": "0"}, "transformation": [{"transformation": "Binarizer", "selectedAsDefault": 1}], "updatedLabel": "BEN_COMP_PAID"}, {"transformationsData": [{"feature_label": "CountryName", "transformation_label": "String Indexer"}], "feature": "CountryName", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "AUSTRALIA", "max": "ZIMBABWE", "missing": "0", "distinct": "40"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "CountryName"}, {"transformationsData": [{"feature_label": "Flag", "transformation_label": "String Indexer"}], "feature": "Flag", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "428", "mean": "", "stddev": "", "min": "https://flagcdn.com/w320/au.png", "max": "https://flagcdn.com/w320/zw.png", "missing": "88", "distinct": "38"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Flag"}]}))

	#transformationPostExecutionHook(visaclassautofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run visaclassHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionClassification(visaclassautofe, ["ben_year_of_birth", "FEIN", "lottery_year", "ben_multi_reg_ind", "NUMBER_OF_BENEFICIARIES", "NUM_OF_EMP_IN_US", "gender_stringindexer", "employer_name_stringindexer", "mail_addr_stringindexer", "city_stringindexer", "state_stringindexer", "zip_stringindexer", "status_type_stringindexer", "rec_date_stringindexer", "first_decision_date_stringindexer", "REQUESTED_CLASS_stringindexer", "BASIS_FOR_CLASSIFICATION_stringindexer", "REQUESTED_ACTION_stringindexer", "BEN_COUNTRY_OF_BIRTH_stringindexer", "BEN_CURRENT_CLASS_stringindexer", "JOB_TITLE_stringindexer", "WORKSITE_ZIP_stringindexer", "valid_from_stringindexer", "valid_to_stringindexer", "BEN_EDUCATION_CODE_stringindexer", "ED_LEVEL_DEFINITION_stringindexer", "BEN_PFIELD_OF_STUDY_stringindexer", "BEN_COMP_PAID_binarizer", "CountryName_stringindexer", "Flag_stringindexer"], "FIRST_DECISION_stringindexer")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    Accuracy = np.round((100 * sklearn.metrics.accuracy_score(y_true=y_test, y_pred=y_predicted)), 1)
    F1= np.round(
            (100 * sklearn.metrics.f1_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Precision= np.round((
                100 * sklearn.metrics.precision_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Recall = np.round((
                100 * sklearn.metrics.recall_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    display(" Accuracy of Prediction on test data    : %s"%Accuracy)
    display(" F1 score of Prediction on test data    : %s"%F1)
    display(" Precision of Prediction on test data   : %s"%Precision)
    display(" Recall of Prediction on test data      : %s"%Recall)
    display(df.head())
except Exception as ex:
    logging.error(ex)

spark.stop()

